# SQL: Исследование приложения о книгах

**Цель исследования**: проанализировать базу данных и получить информацию, которая впоследствии поможет сформулировать ценностное предложение для нового продукта.

**Для анализа доступна** база данных, содержащая информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Шаги исследования**:
1. Подключение к базе данных
2. Получение данных по SQL-запросам
3. Выводы

## Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Исследуем таблицы и выведем первые строки

**Таблица `books`**

In [3]:
query = '''
        SELECT *
        FROM books
        '''

In [4]:
pd.io.sql.read_sql(query, con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


В таблице `books` содержатся данные о книгах: id книги, id автора, название, число страниц, дата публикации, id издателя

**Таблица `authors`**

In [5]:
query = '''
        SELECT *
        FROM authors
        '''

In [6]:
pd.io.sql.read_sql(query, con = engine).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


В таблице `authors` содержатся данные об авторах: id и имя автора

**Таблица `ratings`**

In [7]:
query = '''
        SELECT *
        FROM ratings
        '''

In [8]:
pd.io.sql.read_sql(query, con = engine).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


В таблице `ratings` содержатся данные о пользовательских оценках книг: id оценки, id книги, имя пользователя и оценка.

**Таблица `publishers`**

In [9]:
query = '''
        SELECT *
        FROM publishers
        '''

In [10]:
pd.io.sql.read_sql(query, con = engine).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


В таблице `publishers` содержатся данные об издательствах: id и название издательства.

**Таблица `reviews`**

In [11]:
query = '''
        SELECT *
        FROM reviews
        '''

In [12]:
pd.io.sql.read_sql(query, con = engine).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


В таблице `reviews` содержатся данные о пользовательских обзорах: id обзора, id книги, имя пользователя и текст обзора.

## Получение данных по SQL-запросу

### Задание 1: *Посчитайте, сколько книг вышло после 1 января 2000 года*

In [13]:
query_1 = '''
        SELECT COUNT(book_id)
        FROM books
        WHERE publication_date > '2000-01-01'
        '''

In [14]:
pd.io.sql.read_sql(query_1, con = engine)

,count
0,819


После 01.01.2000 года вышло 819 книг

### Задание 2: *Для каждой книги посчитайте количество обзоров и среднюю оценку*

In [15]:
query_2 = '''
        WITH x AS
        (SELECT b.book_id, title, COUNT(review_id) AS review_quantity
        FROM books AS b 
        LEFT JOIN reviews AS re ON b.book_id = re.book_id
        GROUP BY b.book_id)
        SELECT title, review_quantity, AVG(rating) AS average_rating
        FROM x LEFT JOIN ratings AS r
        ON x.book_id = r.book_id
        GROUP BY title, review_quantity
        ORDER BY review_quantity DESC
        '''

In [16]:
pd.io.sql.read_sql(query_2, con = engine)

,title,review_quantity,average_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,The Da Vinci Code (Robert Langdon #2),6,3.830508
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
4,The Catcher in the Rye,6,3.825581
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
997,The Natural Way to Draw,0,3.000000
998,Leonardo's Notebooks,0,4.000000


Получена информация о количестве обзоров и средней оценке для каждой из книг.

### Задание 3: *Определите издательство, которое выпустило наибольшее число книг толще 50 страниц*

In [17]:
query_3 = '''
        SELECT publisher
        FROM books AS b 
        JOIN publishers AS p ON b.publisher_id = p.publisher_id
        WHERE num_pages > 50
        GROUP BY p.publisher_id
        ORDER BY COUNT(b.book_id) DESC
        LIMIT 1
        '''

In [18]:
pd.io.sql.read_sql(query_3, con = engine)

,publisher
0,Penguin Books


Наибольшее количество книг толще 50 страниц выпустило издательство `Penguin Books`.

### Задание 4: *Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками*

In [19]:
query_4 = '''
        WITH x AS
        (SELECT b.book_id, title, COUNT(rating) AS rating_count, author_id
        FROM books AS b 
        JOIN ratings AS r ON b.book_id = r.book_id
        GROUP BY b.book_id
        HAVING COUNT(rating) >= 50)
        SELECT author, ROUND(AVG(rating), 2)
        FROM authors AS a RIGHT JOIN x on a.author_id = x.author_id
        JOIN ratings AS r ON x.book_id = r.book_id
        GROUP BY a.author_id
        ORDER BY AVG(rating) DESC
        LIMIT 1
        '''

In [20]:
pd.io.sql.read_sql(query_4, con = engine)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


Самая высокая средняя оценка у авторов `J.K. Rowling/Mary GrandPré`. Это логично, рейтинги книг о Гарри Поттере всегда на высоте.

### Задание 5: *Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок*

In [21]:
query_5 = '''
        WITH x AS
        (SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(rating) > 50),
        y AS 
        (SELECT COUNT(review_id) AS review_quantity
        FROM reviews AS r
        JOIN x ON r.username = x.username
        GROUP BY x.username)
        SELECT ROUND(AVG(review_quantity))
        FROM y
        '''

In [22]:
pd.io.sql.read_sql(query_5, con = engine)

,round
0,24.0


В среднем пользователи с более чем 50 оценками пишут около 24 обзоров

## Выводы

В ходе исследования проанализирована информация, содержащаяся в базе данных сервиса для чтения книг по подписке.

Выполнен ряд запросов к базе данных, по результатам которых даны ответы на основные интересующие вопросы:
1. *После 01.01.2000 года вышло 819 книг*
2. *Выведена информация о количестве обзоров и средней оценке для каждой из книг*
3. *Наибольшее количество книг толще 50 страниц выпустило издательство Penguin Books*
4. *Самая высокая средняя оценка у авторов J.K. Rowling/Mary GrandPré*
5. *В среднем пользователи с более чем 50 оценками пишут около 24 обзоров*